In [34]:
import numpy as np
import pandas as pd 
import re 
import nltk
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\resala\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
df = pd.read_csv(r'E:\NLP_200\Code Section 05\amazonreviews.tsv', sep='\t')
df.head()


,label,review
0,pos,Stuning even for the non-gamer: This sound tra...
1,pos,The best soundtrack ever to anything.: I'm rea...
2,pos,Amazing!: This soundtrack is my favorite music...
3,pos,Excellent Soundtrack: I truly like this soundt...
4,pos,"Remember, Pull Your Jaw Off The Floor After He..."


In [10]:
df.label.value_counts()

neg    5097
pos    4903
Name: label, dtype: int64

In [11]:
len(df)

10000

In [12]:
df.isnull().sum()

label     0
review    0
dtype: int64

In [36]:
documents = []

from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()

for sen in range(0,len(df['review'])):
       # Remove all the special characters
    document = re.sub(r'\W', ' ', str(df['review'][sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    #document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(documents).toarray()
y=df['label']

In [ ]:
#pd.DataFrame(documents)
#x= documents
#y=df['label']

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print('Training Data Shape:' , X_train.shape)
print('Training Data Shape:' , X_test.shape)

Training Data Shape: (6700, 1500)
Training Data Shape: (3300, 1500)


In [16]:
y_train.value_counts()

neg    3448
pos    3252
Name: label, dtype: int64

In [23]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=1000, random_state=42)
classifier.fit(X_train, y_train) 

RandomForestClassifier(n_estimators=1000, random_state=42)

In [24]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[1329  320]
 [ 272 1379]]
              precision    recall  f1-score   support

         neg       0.83      0.81      0.82      1649
         pos       0.81      0.84      0.82      1651

    accuracy                           0.82      3300
   macro avg       0.82      0.82      0.82      3300
weighted avg       0.82      0.82      0.82      3300

0.8206060606060606
